Notebook which processes the h5 data file produced for a timescan at the beamline XPP at the Linear Coherent Light Source. The notebook outputs the average region of interest intensity for the laser on and laser off shots of the timescan

In [ ]:
### import modules
import h5py
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy import signal
from tempfile import TemporaryFile

In [ ]:
### import the h5 file
f = h5py.File('/data3/2021/2021_3_LCLS_LAO/smalldata/xpplv7918_Run0033.h5','r')

In [ ]:
#### extract data fields
ipm2 = np.array(f['ipm2/sum'])
detec = np.array(f['jungfrau1M/ROIw_0_area'])
WP = np.array(f['epics/las_comp_wp'])
xOn = np.array(f['lightStatus/xray'])
laserOn = np.array(f['lightStatus/laser'])
lasdelay = np.array(f['enc/lasDelay'])
theta = np.array(f['epics/gon_kappa_eta'])
tt = np.array(f['tt/FLTPOS_PS'])

In [ ]:
### time delay data field
timeDelay = np.array(f['scan/lxt_ttc'])*1e12 

In [ ]:
### time delay bin size 
tauBin = np.nanmin([ ((np.nanmax(timeDelay)-np.nanmin(timeDelay))/(np.unique(timeDelay).size - 1)), 1e9])

In [ ]:
### define a detector threshold and sort shots into laser on and laser off
thresh = 20.0
ind_off = np.where(np.logical_and(np.logical_and(xOn==1, laserOn==0), ipm2>thresh))
ind_on = np.where(np.logical_and(np.logical_and(xOn==1, laserOn==1), ipm2>thresh))

In [ ]:
### define time delays and sort shots into laser on and laser off
delays_on = timeDelay[ind_on]
tt_on = tt[ind_on]
ipm_on = ipm2[ind_on]

ipm_off = ipm2[ind_off]
tt_off = tt[ind_off]
delays_off = timeDelay[ind_off]

### range of the scan: -1ps to 20 ps. 60 steps gives the bin size
tau = np.linspace(-1, 20, 60) 
taubin = np.diff(tau)[0]

tau_Delays = tau[:-1] + np.diff(tau)/2.
tau_Delays = np.insert(tau_Delays, 0, tau_Delays[0] - taubin)
tau_Delays = np.append(tau_Delays, tau_Delays[len(tau_Delays)-1] + taubin)

detec_off = detec_roi[ind_off]
detec_on = detec_roi[ind_on]

delays_on_indices = np.arange(0, len(tau_Delays), 1)
delays_off_indices = np.arange(0, len(tau_Delays), 1)

In [ ]:
%matplotlib notebook

totalArray_on = np.zeros((len(tau_Delays), detec_on.shape[1], detec_on.shape[2]))#len(tau_Delays))
totalArray_off = np.zeros((len(tau_Delays), detec_off.shape[1], detec_off.shape[2]))

meanArray_on = np.zeros(len(tau_Delays))
meanArray_off = np.zeros(len(tau_Delays))

### calculating the average detector value for laser on
for index in range(len(totalArray_on)):
    avg_ipm_on = np.mean(ipm_on[np.where(np.digitize(delays_on+tt_on, bins = tau) == delays_on_indices[index])])
    totalArray_on[index]=np.mean(detec_on[np.where(np.digitize(delays_on+tt_on, bins = tau) == delays_on_indices[index])], axis=0)/avg_ipm_on
    meanArray_on[index] = np.mean(detec_on[np.where(np.digitize(delays_on+tt_on, bins = tau) == delays_on_indices[index])])/avg_ipm_on

### calculating the average detector value for laser off
for index1 in range(len(totalArray_off)):
    avg_ipm_off = np.mean(ipm_off[np.where(np.digitize(delays_off+tt_off, bins = tau) == delays_off_indices[index1])])
    totalArray_off[index1]=np.mean(detec_off[np.where(np.digitize(delays_off+tt_off, bins = tau) == delays_off_indices[index1])], axis=0)/avg_ipm_off
    meanArray_off[index1]=np.mean(detec_off[np.where(np.digitize(delays_off+tt_off, bins = tau) == delays_off_indices[index1])])/avg_ipm_off


    

In [ ]:
### plot the timedelay scans for laser on and laser off
%matplotlib notebook

meanArray_off[16] = meanArray_off[15]#np.nan
meanArray_off[28] = meanArray_off[27]#np.nan
meanArray_off[37] = meanArray_off[36]#np.nan
meanArray_off[59] = meanArray_off[60]#np.nan

plt.plot(tau_Delays, meanArray_on, '-o', color='black')
plt.plot(tau_Delays, meanArray_off, '-o', color='red')
plt.xlabel('time delay (ps)')
plt.ylabel('normalized intensity')

totalArray_off[16] = totalArray_off[15]#np.nan
totalArray_off[28] = totalArray_off[27]#np.nan
totalArray_off[37] = totalArray_off[36]#np.nan
totalArray_off[59] = totalArray_off[60]#np.nan

#plt.savefig('/home/jg2364/lcls_lao_analysis/figs/40_good.pdf')

In [ ]:
### save the laser on rocking curve
with open('33Array_On', 'wb') as f:
    np.save(f, totalArray_on)

In [ ]:
### save the laser off rocking curve
with open('33Array_Off', 'wb') as f:
    np.save(f, totalArray_off)